# S3 Access Single-Pol Examples

This notebook uses `nisarh5toimage` to extract data from various images stored in s3 buckets.

In [1]:
!nisarh5toimage --help

usage: nisarh5toimage [-h] [--info] [--quickLook] [--ros3] [--conserveMem]
                      [--productFamily {RSLC,ROFF,RIFG,RUNW,GCOV,GUNW,GOFF}]
                      [--frequencyB] [--dB] [--sigma0] [--scaleToPixels]
                      [--wrapped] [--polarization {HH,VV,HV,VH}]
                      [--downsampleFactor DOWNSAMPLEFACTOR]
                      [--fields FIELDS [FIELDS ...]]
                      [--layers {1,2,3} [{1,2,3} ...]]
                      [--outputFormat {COG,GTiff,binary}]
                      productName [output]

Convert NISAR H5 product to Tiff(s), COG(s) or binary files

positional arguments:
  productName           NISAR h5 product (file path, s3, or https)
  output                Root name for ouput. Band (e.g., .unwrappedPhase) and
                        file type (e.g., .tiff) will be appended. (Not
                        required for --info)

options:
  -h, --help            show this help message and exit
  --info                Print 

## File Locations

In [2]:
tmpOut = '/home/jovyan/tmpRSLC'

In [3]:
SHFile = 's3://nisar-adt-batch/commissioning/adt-on-demand/products/L1_L_RSLC/2025/09/20/NISAR_L1_PR_RSLC_005_057_A_001_7700_SHNA_A_20250920T064630_20250920T064704_A00408_P_P_J_100/NISAR_L1_PR_RSLC_005_057_A_001_7700_SHNA_A_20250920T064630_20250920T064704_A00408_P_P_J_100.h5'

## Case 1: Header Only

This command just reads the header info and prints a quick summary. By default it uses `ros3`.

In [4]:
!nisarh5toimage {SHFile} --info

page_buf_size 2147.483648 MB
Opening s3://nisar-adt-batch/commissioning/adt-on-demand/products/L1_L_RSLC/2025/09/20/NISAR_L1_PR_RSLC_005_057_A_001_7700_SHNA_A_20250920T064630_20250920T064704_A00408_P_P_J_100/NISAR_L1_PR_RSLC_005_057_A_001_7700_SHNA_A_20250920T064630_20250920T064704_A00408_P_P_J_100.h5 with ros3
loading as power False
Data loaded

Default polarization:
HH 
Available fields: 
HH

Select Parameters
RSLC.track = 57 
RSLC.referenceOrbit = 749 
RSLC.frame = 1 
RSLC.datetime = 2025-09-20 06:46:30 
RSLC.centerLat = 66.15927799537198 
RSLC.centerLon = -43.65223420121839 
RSLC.referenceGranule = None
RSLC.rangeBandwidth = 77000000.0 
RSLC.SLCNearRange = 895505.0547508333 
RSLC.SLCFarRange = 980293.2321170624 
RSLC.SLCFirstZeroDopplerTime = 24390.0 
RSLC.SLCLastZeroDopplerTime = 24424.999342105264 
RSLC.LookDirection = left 
RSLC.PassType = ascending 
RSLC.Wavelength = 0.23840354512922465 
RSLC.PRF = 1520.0 
RSLC.epsg = 4326 
RSLC.NumberRangeLooks = 1 
RSLC.NumberAzimuthLooks = 1

## Case 2 Read SH band using Memory and Ros3

For this case, there are four possible ways to read the hdf5 file:
1) Stream file to memory and convert the data to numpy arrays,
2) Use `ros3` and convert to numpy arrays,
3) Stream file to memory, and write the data in blocks so no full numpy array is generated, and
4) Use `ros3` and block writes.

This example essentially reads all of the data in the file, so the advantages of cloud-optimized hdf are less apparent.

### 1) Stream to Memory, Convert to Numpy

By default, the files is streamed to memory (see [codeblock](https://github.com/fastice/nisarhdf/blob/57648610195d7b345ebe2779f4e7552d3d8ce332/nisarhdf/nisarBaseHDF.py#L506)). In this case, there should be no network penalty because all case must read the entire file.

In [5]:
!nisarh5toimage {SHFile} {tmpOut} --outputFormat binary #--downsampleFactor 2

[INFO] Downloading s3://nisar-adt-batch/commissioning/adt-on-demand/products/L1_L_RSLC/2025/09/20/NISAR_L1_PR_RSLC_005_057_A_001_7700_SHNA_A_20250920T064630_20250920T064704_A00408_P_P_J_100/NISAR_L1_PR_RSLC_005_057_A_001_7700_SHNA_A_20250920T064630_20250920T064704_A00408_P_P_J_100.h5 into memory...
[INFO] File downloaded into memory, opening with h5py...
loading as power False
Data loaded

Run time: 
Load data: 0:02:11.140412
Write data: 0:01:10.944294
Total 0:03:22.084706

Network traffic: 
Bytes sent:   32.5M
Bytes received:   10.5G

Memory Usage: 
Current memory usage:   24.9M
Peak memory usage:   54.6G


### 2) Use Ros 3, Convert to Numpy

Here is the same command again only using `ros3` (see [codeblock](https://github.com/fastice/nisarhdf/blob/57648610195d7b345ebe2779f4e7552d3d8ce332/nisarhdf/nisarBaseHDF.py#L546)). Using `ros3` is little less than 2x slower, but saves memory by not having to buffer the whole h5 file. 

In [6]:
!nisarh5toimage {SHFile} {tmpOut} --outputFormat binary --ros3

page_buf_size 2147.483648 MB
Opening s3://nisar-adt-batch/commissioning/adt-on-demand/products/L1_L_RSLC/2025/09/20/NISAR_L1_PR_RSLC_005_057_A_001_7700_SHNA_A_20250920T064630_20250920T064704_A00408_P_P_J_100/NISAR_L1_PR_RSLC_005_057_A_001_7700_SHNA_A_20250920T064630_20250920T064704_A00408_P_P_J_100.h5 with ros3
loading as power False
Data loaded

Run time: 
Load data: 0:04:41.312729
Write data: 0:00:59.469616
Total 0:05:40.782345

Network traffic: 
Bytes sent:   59.1M
Bytes received:   10.6G

Memory Usage: 
Current memory usage:    8.5M
Peak memory usage:   43.1G


### 3) Stream to Memory, No Numpy (--conserveMem)

This case reduces memory usage by eliminating the large numpy array and block decoding the h5 file from memory. This case is faster and and more memory efficient than the two methods above.

In [11]:
!nisarh5toimage {SHFile} {tmpOut} --outputFormat binary --conserveMem

[INFO] Downloading s3://nisar-adt-batch/commissioning/adt-on-demand/products/L1_L_RSLC/2025/09/20/NISAR_L1_PR_RSLC_005_057_A_001_7700_SHNA_A_20250920T064630_20250920T064704_A00408_P_P_J_100/NISAR_L1_PR_RSLC_005_057_A_001_7700_SHNA_A_20250920T064630_20250920T064704_A00408_P_P_J_100.h5 into memory...
[INFO] File downloaded into memory, opening with h5py...
loading as power False
Data loaded
Writing 53200 rows in blocks of 4608 rows (aligned to 512)...
Wrote rows 50688-53200 (100.0%)
Done.

Run time: 
Load data: 0:00:43.470192
Write data: 0:01:28.893644
Total 0:02:12.363836

Network traffic: 
Bytes sent:   29.4M
Bytes received:   10.5G

Memory Usage: 
Current memory usage:   24.9M
Peak memory usage:   14.4G


### 4) Ros 3, no Numpy

With intermediate numpy array and ros3, the memory usage is reduced greatly. This case takes about twice the time as the stream to memory case above, but is much more memory efficient.

In [8]:
!nisarh5toimage {SHFile} {tmpOut} --outputFormat binary --ros3 --conserveMem

page_buf_size 2147.483648 MB
Opening s3://nisar-adt-batch/commissioning/adt-on-demand/products/L1_L_RSLC/2025/09/20/NISAR_L1_PR_RSLC_005_057_A_001_7700_SHNA_A_20250920T064630_20250920T064704_A00408_P_P_J_100/NISAR_L1_PR_RSLC_005_057_A_001_7700_SHNA_A_20250920T064630_20250920T064704_A00408_P_P_J_100.h5 with ros3
loading as power False
Data loaded
Writing 53200 rows in blocks of 4608 rows (aligned to 512)...
Wrote rows 50688-53200 (100.0%)
Done.

Run time: 
Load data: 0:00:01.622946
Write data: 0:04:23.202445
Total 0:04:24.825391

Network traffic: 
Bytes sent:   54.0M
Bytes received:   10.5G

Memory Usage: 
Current memory usage:    8.5M
Peak memory usage:    3.8G
